# Paqutes

In [18]:
!pip install -qU langchain-openai
# !pip install langchain==0.3
# !pip install langchain-community==0.2.16
!pip install FPDF
!pip install pypdf
!pip install llama-index
!pip install neo4j
!pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j

In [72]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_community.graphs import Neo4jGraph
from langchain_core.documents import Document
from langchain_community.graph_vectorstores.extractors import (
    KeybertLinkExtractor,
    LinkExtractorTransformer,
)
from langchain.chains import GraphCypherQAChain

from fpdf import FPDF
import getpass
import os
import networkx as nx
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import KnowledgeGraphRAGRetriever
from langchain_openai import OpenAIEmbeddings
from langchain_community.graph_vectorstores import CassandraGraphVectorStore
# from llama_index.core.response_synthesis import ResponseSynthesizer
# import matplotlib.pyplot as plt

# Inicio el LLMs y creo el graph

In [8]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

 ········


In [27]:
# Initialize LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature = 0)

In [28]:
llm_transformer = LLMGraphTransformer(llm=llm)

# Cargo y preproceso el texto

In [47]:
text = """Harry Potter is a wizard who studies at Hogwarts School of 
Witchcraft and Wizardry. His best friends are Hermione Granger and Ron Weasley. Harry is in Gryffindor House, 
just like Ron and Hermione. He often faces challenges from Draco Malfoy, who belongs to Slytherin House. Harry's parents,
Lily and James Potter, were killed by Lord Voldemort, a dark wizard. Harry is known for defeating Voldemort as a baby and later becoming the owner 
of the Elder Wand."""

In [48]:
documents = [Document(page_content=text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [55]:
print(f"Nodes:{graph_documents[0].nodes}\n")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='Harry Potter', type='Person', properties={}), Node(id='Wizard', type='Occupation', properties={}), Node(id='Hogwarts School Of Witchcraft And Wizardry', type='School', properties={}), Node(id='Hermione Granger', type='Person', properties={}), Node(id='Ron Weasley', type='Person', properties={}), Node(id='Gryffindor House', type='House', properties={}), Node(id='Draco Malfoy', type='Person', properties={}), Node(id='Slytherin House', type='House', properties={}), Node(id='Lily Potter', type='Person', properties={}), Node(id='James Potter', type='Person', properties={}), Node(id='Lord Voldemort', type='Person', properties={}), Node(id='Elder Wand', type='Object', properties={})]

Relationships:[Relationship(source=Node(id='Harry Potter', type='Person', properties={}), target=Node(id='Wizard', type='Occupation', properties={}), type='HAS_OCCUPATION', properties={}), Relationship(source=Node(id='Harry Potter', type='Person', properties={}), target=Node(id='Hogwarts School O

In [58]:
graph_documents[0].relationships[0]

Relationship(source=Node(id='Harry Potter', type='Person', properties={}), target=Node(id='Wizard', type='Occupation', properties={}), type='HAS_OCCUPATION', properties={})

# Store Graph

In [68]:
os.environ["NEO4J_URI"]='neo4j+s://1a2965d6.databases.neo4j.io'
os.environ["NEO4J_USERNAME"]='neo4j'
os.environ["NEO4J_PASSWORD"]='Zifh1asGv3erUqx6F9TNj93HixcqvOLqJTrCjA68F6E'
# os.environ["AURA_INSTANCEID"]='1a2965d6'
# os.environ["AURA_INSTANCENAME"]='Instance01'
graph = Neo4jGraph(refresh_schema=False)

In [69]:
graph.add_graph_documents(graph_documents)

In [70]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Person {id: STRING}
Occupation {id: STRING}
School {id: STRING}
House {id: STRING}
Object {id: STRING}
Relationship properties:

The relationships:
(:Person)-[:HAS_OCCUPATION]->(:Occupation)
(:Person)-[:STUDIES_AT]->(:School)
(:Person)-[:HAS_FRIEND]->(:Person)
(:Person)-[:BELONGS_TO]->(:House)
(:Person)-[:RIVAL_OF]->(:Person)
(:Person)-[:PARENT_OF]->(:Person)
(:Person)-[:OWNS]->(:Object)
(:Person)-[:KILLED_BY]->(:Person)


# Chain

In [74]:
chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True, allow_dangerous_requests = True)

In [75]:
chain.invoke({"query": "Who are Harry Potter Friends. Show me how you conclude that"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {id: "Harry Potter"})-[:HAS_FRIEND]->(friend)
RETURN friend
Full Context:
[{'friend': {'id': 'Hermione Granger'}}, {'friend': {'id': 'Ron Weasley'}}]

> Finished chain.


{'query': 'Who are Harry Potter Friends. Show me how you conclude that',
 'result': "Harry Potter's friends are Hermione Granger and Ron Weasley."}

In [76]:
chain.invoke({"query": "Who is Federer"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {id: 'Federer'}) RETURN p
Full Context:
[]

> Finished chain.


{'query': 'Who is Federer', 'result': "I don't know the answer."}